## Overview

This pose_estimation sample application detects people in camera streams, and estimates pose of each person. To achieve this, this sample application uses 2 types of models, 1) people detection model, and 2) pose estimation models.

This notebook explains how to build and deploy the application step by step.

#### How this application works

1. Apply people detection model to camera image.
2. Choose up to 4 boxes based on the people detection scores.
3. Transform the bounding boxes and sub-images to the input shape of the pose estimation model.
4. Apply pose estimation model to the sub images. Use one of 4 variations of pose estimation models based on the number of sub-images (1~4).
5. Find 17 joint positions for each person, from the output from the pose estimation model.



## Prerequisites

Before you start processing this notebook, some prerequisites need to be completed.

* Set up your AWS Panorama Appliance - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-setup.html)
* Create at least one camera stream - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-setup.html#gettingstarted-setup-camera)
* Create an IAM Role for your application - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/permissions-application.html)
* Install "panorama-cli" tool [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-deploy.html#gettingstarted-deploy-prerequisites)

## Import libraries and define configurations

First step is to import all libraries needed.

In [ ]:
import sys
import os
import time
import tarfile
import json
import uuid

import boto3
import gluoncv

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility


You need to specify some information specific to your environment.

In [ ]:
account_id = input( "Your AWS account ID (format : 12 digits)" ).strip()
region_name = input("Region name (e.g. us-east-1)").strip()
s3_bucket = input( "Your S3 bucket for model compilation" ).strip()

# Following configurations are required when you use real hardware, 
# thus can be any dummy strings when you use only Test Utility.
device_id = input("Device Id (format : device-*)").strip()
runtime_role_arn = input("IAM Role ARN for the application on the device").strip()
data_source_name = input( "Your data source (Camera) name" ).strip()

## Import application

With "panorama-cli import-application" command, replacing placeholder information in application files. This step essentially replace placeholder ("123456789012") with your aws account id.

In [ ]:
!cd ./pose_estimation_app/ && panorama-cli import-application

## Prepare models

#### Export models from GluonCV model zoo

Exporting models from GluonCV's model zoo. This sample uses "yolo3_mobilenet1.0_coco" for people detection, and "simple_pose_resnet152_v1d" for pose estimation.

In [ ]:
def export_model_and_create_targz( prefix, name, model ):
    os.makedirs( prefix, exist_ok=True )
    gluoncv.utils.export_block( os.path.join( prefix, name ), model, preprocess=False, layout="CHW" )

    tar_gz_filename = f"{prefix}/{name}.tar.gz"
    with tarfile.open( tar_gz_filename, "w:gz" ) as tgz:
        tgz.add( f"{prefix}/{name}-symbol.json", f"{name}-symbol.json" )
        tgz.add( f"{prefix}/{name}-0000.params", f"{name}-0000.params" )
        
    print( f"Exported : {tar_gz_filename}" )

In [ ]:
# Export object detection model. Reset the classes for human detection only.
people_detection_model = gluoncv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
people_detection_model.reset_class(["person"], reuse_weights=['person'])
export_model_and_create_targz( "models", "yolo3_mobilenet1.0_coco_person", people_detection_model )

# Export pose estimation model.
pose_estimation_model = gluoncv.model_zoo.get_model('simple_pose_resnet152_v1d', pretrained=True)
export_model_and_create_targz( "models", "simple_pose_resnet152_v1d", pose_estimation_model )

#### Add exported model files in the model package

With "panorama-cli add-raw-model" command, Adding exported models into a model package.

Please note that we use 4 variations of simple_pose_resnet152_v1d models with different input data shapes ( [1, 3, 256, 192] ~ [4, 3, 256, 192] ), so there are 5 models in total.

In [ ]:
!cd ./pose_estimation_app/ && panorama-cli add-raw-model \
    --model-asset-name people_detection_model \
    --model-local-path ../models/yolo3_mobilenet1.0_coco_person.tar.gz \
    --descriptor-path packages/{account_id}-pose_estimation_models-1.0/descriptor_people_detection.json \
    --packages-path packages/{account_id}-pose_estimation_models-1.0

In [ ]:
for num_batch_images in range(1,5):
    !cd ./pose_estimation_app/ && panorama-cli add-raw-model \
        --model-asset-name pose_estimation_model_{num_batch_images} \
        --model-local-path ../models/simple_pose_resnet152_v1d.tar.gz \
        --descriptor-path packages/{account_id}-pose_estimation_models-1.0/descriptor_pose_estimation_{num_batch_images}.json \
        --packages-path packages/{account_id}-pose_estimation_models-1.0
    
    print("---")

## Prepare business logic

#### Preview python source code
Next step is to build a business logic container. This application's business logic consists of single python source code. Let's preview it.

In [ ]:
panorama_test_utility.preview_text_file( f"./pose_estimation_app/packages/{account_id}-pose_estimation_code-1.0/src/app.py" )

#### Test run the business logic with test-utility

Let's run the application with Test Utility, with following steps.

1. Compile models with Test Utility **Compile** command.
2. Run application with Test Utility **Run** command, and check if it runs as expected.

To iterate the development, you can edit the python source code with your prefered text editor (or even within Jupyter environment), and re-run the application.

In [ ]:
# Compile the models to run with test-utility.
# Specifying 5 sets of model related arguments to compile 5 models.
# This step takes 40 mins ~ 60 mins.

people_detection_model_data_shape = '{"data":[1,3,480,600]}'

pose_estimation_model_data_shapes = [
    '{"data":[1,3,256,192]}',
    '{"data":[2,3,256,192]}',
    '{"data":[3,3,256,192]}',
    '{"data":[4,3,256,192]}',
]

%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{s3_bucket}/pose_estimation_app \
\
--model-node-name people_detection_model \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
--model-data-shape '{people_detection_model_data_shape}' \
--model-framework MXNET \
\
--model-node-name pose_estimation_model_1 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
--model-data-shape '{pose_estimation_model_data_shapes[0]}' \
--model-framework MXNET \
\
--model-node-name pose_estimation_model_2 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
--model-data-shape '{pose_estimation_model_data_shapes[1]}' \
--model-framework MXNET \
\
--model-node-name pose_estimation_model_3 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
--model-data-shape '{pose_estimation_model_data_shapes[2]}' \
--model-framework MXNET \
\
--model-node-name pose_estimation_model_4 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
--model-data-shape '{pose_estimation_model_data_shapes[3]}' \
--model-framework MXNET

In [ ]:
# Run the application with test-utility.
#
# As '--output-screenshots' option is specified, this command simulates HDMI output by generating sequentially numbered screenshot files.
# You can find screenshot files under ./screenshot directory.

video_filepath = "../common/test_utility/videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
--app-name pose_estimation_app \
--code-package-name pose_estimation_code \
--model-package-name pose_estimation_models \
--camera-node-name abstract_rtsp_media_source \
\
--model-node-name people_detection_model \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--model-node-name pose_estimation_model_1 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
\
--model-node-name pose_estimation_model_2 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
\
--model-node-name pose_estimation_model_3 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
\
--model-node-name pose_estimation_model_4 \
--model-file-basename ./models/simple_pose_resnet152_v1d \
\
--video-file {video_filepath} \
--video-start 0 \
--video-stop 30 \
--video-step 1 \
\
--output-screenshots ./screenshot/%Y%m%d-%H%M%S \
\
--py-file ./pose_estimation_app/packages/{account_id}-pose_estimation_code-1.0/src/app.py

In [ ]:
import glob
import IPython

latest_screenshot_dirname = sorted( glob.glob( "./screenshot/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

#### Build application logic container

With "panorama-cli build-container" command, building a container image, and add it into the "pose_estimation_code" package.

This step takes long time (5~10 mins), and because it is using %%capture magic command, you don't see progress during the process. Please wait.

In [ ]:
%%capture captured_output
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./pose_estimation_app && panorama-cli build-container \
    --container-asset-name code \
    --package-path packages/{account_id}-pose_estimation_code-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

## Package application (upload locally prepared packages onto Cloud)

Now you have prepared both model packages and code package locally. Let's upload those packages to the cloud with "panorama-cli package-application" command.

In [ ]:
!cd ./pose_estimation_app && panorama-cli package-application

## Deploy application to the device programatically

Once you uploaded the packages to the cloud, you can create an application instance on your device. You need to specify a manifest file, and optionally an override-manifest file. In this sample app, "panorama::abstract_rtsp_media_source" is used as a data source node, and it has to be overridden by override manifest file, so using both files.

You can manually create override-manufest file, but in this sample notebook, we create it programmatically from template file and the Data Source name you input at the beginning.

#### Advanced topic
If you have multiple cameras, you can include them in the "override.json". Please refer to "override_template_multi_cameras.json" how to include multiple cameras in the override maifest file.

In [ ]:
# Read template file
with open( "./pose_estimation_app/graphs/pose_estimation_app/override_template.json", "r" ) as fd:
    data = fd.read()

# Replace placeholders
data = data.replace( "{YourAwsAccountId}", account_id )
data = data.replace( "{YourCameraName}", data_source_name )

# Write override-manifest file
with open( "./pose_estimation_app/graphs/pose_estimation_app/override.json", "w" ) as fd:
    fd.write(data)

#### Preview manifest file("graph.json"), and override-manifest file("override.json")

In [ ]:
panorama_test_utility.preview_text_file( "./pose_estimation_app/graphs/pose_estimation_app/graph.json" )

In [ ]:
panorama_test_utility.preview_text_file( "./pose_estimation_app/graphs/pose_estimation_app/override.json" )

#### Deploy the app using the manifest files

In order to create an application instance, this notebook uses boto3's "panorama" client and create_application_instance() API. (It is also possible to use "aws panorama create-application-instance" command instead.)

In [ ]:
# create a boto3 client to access Panorama service
# FIXME : not using AWS_REGION here, because panorama-cli uses only default region currently.
panorama_client = boto3.client("panorama")

In [ ]:
def deploy_application( application_name, manifest_filename, override_filename ):

    def get_payload_from_json( filename ):
        with open( filename ) as fd:
            
            s = fd.read()
            
            assert "{Your" not in s, "Please replace {YourAwsAccountId} and {YourCameraName} in [%s] with your AWS account id" % (filename)
            
            # validating JSON format and making it compact, by loading and dumping, 
            payload = json.dumps(json.loads(s))
            
            return payload

    manifest_payload = get_payload_from_json( manifest_filename )
    override_payload = get_payload_from_json( override_filename )
    
    response = panorama_client.create_application_instance(
        Name = application_name,
        RuntimeRoleArn = runtime_role_arn,
        DefaultRuntimeContextDevice = device_id,
        ManifestPayload = {"PayloadData":manifest_payload},
        ManifestOverridesPayload = {"PayloadData":override_payload},
    )
        
    return response

In [ ]:
application_name = "pose_estimation_notebook_" + str(uuid.uuid4())[:8]

response = deploy_application(
    application_name = application_name,
    manifest_filename = "./pose_estimation_app/graphs/pose_estimation_app/graph.json",
    override_filename = "./pose_estimation_app/graphs/pose_estimation_app/override.json"
)

application_instance_id = response["ApplicationInstanceId"]

response

In [ ]:
application_name

#### Wait for deployment completion

Application instance creation has been triggered. This notebook checks the progress by calling describe_application_instance() API periodically. Please confirm that you see "DEPLOYMENT_SUCCEEDED" status at the end.

In [ ]:
def wait_deployment( application_instance_id ):
    
    progress_dots = panorama_test_utility.ProgressDots()    
    while True:
        app = panorama_client.describe_application_instance( ApplicationInstanceId = application_instance_id )
        progress_dots.update_status( "%s (%s)" % (app["Status"], app["StatusDescription"]) )
        if app["Status"] not in ( "DEPLOYMENT_PENDING", "DEPLOYMENT_REQUESTED", "DEPLOYMENT_IN_PROGRESS" ):
            break
        time.sleep(60)

wait_deployment( application_instance_id )

#### Visit CloudWatch Logs to check logs from the application instance

If you saw "DEPLOYMENT_SUCCEEDED" status, the application started to run on your device. Application logs are uploaded to CloudWatch Logs. Let's get the URL of CloudWatch Logs management console. "console_output" is the log stream your Python code's stdout/stderr are redirected to.

In [ ]:
logs_url = panorama_test_utility.get_logs_url( region_name, device_id, application_instance_id )
print( "CloudWatch Logs URL :" )
print( logs_url )

## Next steps

If you want to customize this sample application, you can follow these steps:

1. Edit "pose_estimation_app/packages/{account_id}-pose_estimation_code-1.0/src/app.py". You can use your preferred text editor.
2. Test-run the updated script with Test Utility [from here](#Test-run-the-business-logic-with-test-utility). Confirm that the application runs as expected.
3. Follow the steps for the real device ([build new container image](#Build-application-logic-container), [upload the code package](#Package-application-(upload-locally-prepared-packages-onto-Cloud)), and [deploy to the device](#Deploy-the-app-using-the-manifest-files) ). Confirm that the application runs as expected on the device as well.
